In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
"""
        headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

"""

"\n        headers = {\n        'dnt': '1',\n        'upgrade-insecure-requests': '1',\n        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',\n        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',\n        'sec-fetch-site': 'same-origin',\n        'sec-fetch-mode': 'navigate',\n        'sec-fetch-user': '?1',\n        'sec-fetch-dest': 'document',\n        'referer': 'https://www.amazon.com/',\n        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',\n    }\n\n"

In [5]:
headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

In [6]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd

def scrape_amazon_data(product):
    raw_list = []
    final_list = []
    alfa = 1
    product = product.replace(" ", "+")
    max_retries = 3  # Maximum number of retries
    
    while len(raw_list) < 300:
        try:
            if alfa == 1:
                url = "https://www.amazon.in/s?k={}".format(product)
            else:
                url = "https://www.amazon.in/s?k={}&page={}".format(product, alfa)

            # Retry the request multiple times in case of 503 error
            for _ in range(max_retries):
                response = requests.get(url,headers=headers, timeout=10)
                if response.status_code == 200:
                    break  # Break the retry loop if the request is successful
                else:
                    print("Retrying...")

            if response.status_code == 200:
                bsObj = BeautifulSoup(response.content, "html.parser")
                for i in bsObj.findAll("div", {"class": "a-section a-spacing-small puis-padding-left-small puis-padding-right-small"}):
                    raw_list.append(i.text)

                time.sleep(np.random.randint(1, 10))
                alfa += 1
            else:
                print("Failed to fetch the page:", response.status_code)
                break  # Break the main loop if the request fails continuously
        except Exception as e:
            print("Error:", e)
            break  # Break the main loop in case of any other exception

    for alfa in raw_list:
        try:
            target_string = alfa.split("out")
            net_rating = target_string[0].split(' ')[-2]

            if len(net_rating) > 3:
                net_rating = net_rating[-3:]

            title_name = target_string[0].split(' ')[0:-2]
            title_name = " ".join(title_name)

            total_rating = target_string[1].split("stars")
            rating_number = total_rating[1].split(" ")[1]
            price =  total_rating[1].split(" ")[3].split("₹")[1]
            #price = str(price)
            #print([title_name, float(net_rating), int(rating_number.replace(",", "")), float(price.replace(",", ""))])
            final_list.append([title_name, float(net_rating), int(rating_number.replace(",", "")), float(price.replace(",", ""))])
        except:
            print("Error while processing the data")

    return [raw_list,pd.DataFrame(final_list, columns=["title", "net_rating", "rating_number", "price"])]

In [7]:
for i in range(10):
    alfa = scrape_amazon_data("Wilson Super Soft Play Volleyball - Orange/Blue ")
    if(len(alfa) != 0):
        break
alfa

Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Failed to fetch the page: 503
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data
Error while processing the data


[['Wilson Super Soft Play Volleyball - Orange/Blue  4.6 out of 5 stars 790  ₹8,895₹8,895 M.R.P: ₹12,709₹12,709  (30% off)Get it Wednesday, 5 July - Saturday, 8 JulyMore Buying Choices₹8,893(2 new offers)',
  'Wilson WILSO Orange, Blue Super Soft Play  ₹1,249₹1,249 M.R.P: ₹1,299₹1,299  (4% off)Get it by Saturday, 24 June',
  '+1 colors/patternsWilson Soft Play Outdoor Volleyball  4.5 out of 5 stars 18,004  ₹8,814₹8,814 M.R.P: ₹10,999₹10,999  (20% off)Get it Friday, 30 June - Tuesday, 4 July',
  'Boldfit Volleyball Standard Size for Men and Women Sports Volley Ball with Free Pin Strong Grip Vollyball Original Brand Volleyball for Boys Super Volly Ball - Blue Yellow RallyKing- Free Air Pin  4.8 out of 5 stars 80  ₹789₹789 M.R.P: ₹1,599₹1,599  (51% off) FREE Delivery by Amazon',
  'JJ Jonex Super Volley Volleyball Size 4, Blue  4.1 out of 5 stars 17  ₹1,099₹1,099 M.R.P: ₹1,350₹1,350  (19% off) Get it by Thursday, 22 JuneFREE Delivery by AmazonMore Buying Choices₹1,043(3 new offers)',
  'Su

In [8]:
alfa[1]

,title,net_rating,rating_number,price
0,Wilson Super Soft Play Volleyball - Orange/Blue,4.6,790,8895.00
1,+1 colors/patternsWilson Soft Play Outdoor Vol...,4.5,18004,8814.00
2,Boldfit Volleyball Standard Size for Men and W...,4.8,80,789.00
3,"JJ Jonex Super Volley Volleyball Size 4, Blue",4.1,17,1099.00
4,"Cosco BLUYWL-S4 PU Volleyball, Size 4, (Blue Y...",3.5,5,734.00
5,"Cosco Strike, Rubber Machine Stitched Volleyba...",5.0,5,599.00
6,"Nivia Kross Rubber Hand Stitched Volleyball, S...",3.8,4522,489.00
7,"RADION Volleyball Mashi Orange Size 5, Diamete...",2.5,16,419.00
8,Wilson Ace Point Outdoor Volleyball,3.9,9,1349.00
9,Vector X Smash Hand Stitched PU Volleyball (18...,3.8,4,719.00


In [40]:
target_string = alfa[0][0].split("out")
net_rating = target_string[0].split(' ')[-2]

if len(net_rating) > 3:

                
    net_rating = net_rating[-3:]

title_name = target_string[0].split(' ')[0:-2]
title_name = " ".join(title_name)

total_rating = target_string[1].split("stars")
rating_number = total_rating[1].split(" ")[1]
#price = total_rating[1].split("M.R.P:")[1].split(" ")[1].split("₹")[1]

# new price
price =  total_rating[1].split(" ")[3].split("₹")[1]


In [43]:
float(price.replace(",", ""))

8895.0

In [28]:
price =  total_rating[1].split(" ")[3].split("₹")[1]
price

'8,895'

In [14]:
import time
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

def scrape_amazon_data(product):
    raw_list = []
    final_list = []
    alfa = 1
    product = product.replace(" ", "+")
    while len(raw_list) < 300:
        try:
            if alfa == 1:
                url = "https://www.amazon.in/s?k={}".format(product)
            else:
                url = "https://www.amazon.in/s?k={}&page={}".format(product,alfa)

            req = Request(url)
            html = urlopen(req, timeout=10)
            bsObj = BeautifulSoup(html.read(), "html.parser")
            
            for i in bsObj.findAll("div", {"class": "a-section a-spacing-small puis-padding-left-small puis-padding-right-small"}):
                print(i.text)
                raw_list.append(i.text)

            time.sleep(np.random.randint(1, 10))
            alfa += 1
        except:
            print("error")
            pass

    for alfa in raw_list:
        try:
            target_string = alfa.split("out")
            net_rating = target_string[0].split(' ')[-2]

            if len(net_rating) > 3:
                net_rating = net_rating[-3:]

            title_name = target_string[0].split(' ')[0:-2]
            title_name = " ".join(title_name)

            total_rating = target_string[1].split("stars")
            rating_number = total_rating[1].split(" ")[1]
            price = total_rating[1].split("M.R.P:")[1].split(" ")[1].split("₹")[1]
            print([title_name, float(net_rating), int(rating_number.replace(",","")), float(price.replace(",", ""))])
            final_list.append([title_name, float(net_rating), int(rating_number.replace(",","")), float(price.replace(",", ""))])
        except:
            print("error")
            pass
    
    # return dataframe
    return pd.DataFrame(final_list, columns=["title", "net_rating", "rating_number", "price"])


# Call the function to run the web scraping process




In [15]:
scrape_amazon_data("book")

Amazon Brand – Eono Craft Play Creat Handmade Diary with Lock A5 (7x5inch) Diary Unruled 144 Pages  4.1 out of 5 stars 16  100+ viewed in past week₹369₹369 M.R.P: ₹1,499₹1,499  (75% off)
Amazon Brand - Solimo Classic Notebook Checkered  4.2 out of 5 stars 811  300+ viewed in past weekDeal of the Day  ₹219₹219 M.R.P: ₹499₹499  (56% off)
Amazon Brand - Solimo Long Board Book, Alphabets  by Solimo4.4 out of 5 stars 286  Hardcover ₹44.07₹44.07 M.R.P: ₹199₹199  (78% off)
Amazon Brand – Eono Vintage Leather Diary with Key, Leather Bound Journal for Large Writing Notebook 200 Handmade Pages with 5 x 7 inch  4.3 out of 5 stars 116  ₹448₹448 M.R.P: ₹1,499₹1,499  (70% off)
Amazon Brand - Solimo Sketching Book, 100 Sheets, A4 Size  4.6 out of 5 stars 204  Limited time deal  ₹329₹329 M.R.P: ₹700₹700  (53% off)
Amazon Brand - Solimo My Very First Foam Books (Set of 3, Alphabets, Know your Body, Animals & Birds)  by Amazon Brand - Solimo4.0 out of 5 stars 157  Board book ₹315₹315 M.R.P: ₹450₹450  (3

error
error
error
error
error
error
error
error
error


<h1> stress testing </h1>

In [15]:
product_list = ["oil",'football','bag']
for i in product_list:
    scrape_amazon_data(i)

Fortune Sunlite Refined Sunflower Oil, 1L  Sunflower4.4 out of 5 stars 27,824  8K+ bought in past week₹152₹152 (₹152/l) M.R.P: ₹155₹155  (2% off)Buy 3 item(s), get extra 3% offGet it Wednesday, 21 June - Friday, 23 JuneMore Buying Choices₹150(12 new offers)
Fortune Oil, 1 L Pouch Premium Kachi Ghani Pure Mustard  Mustard4.4 out of 5 stars 25,100  6K+ bought in past week₹151₹151 (₹151/l) M.R.P: ₹190₹190  (21% off)  FREE delivery as fast as 4 hours on orders above ₹499 of Fresh items
Fortune Sunlite Refined Sunflower Oil, 5L Can  sunflower4.4 out of 5 stars 12,490  2K+ bought in past week₹840₹840 (₹168/l) M.R.P: ₹925₹925  (9% off) Get it by Wednesday, 21 June
Amazon Brand - Solimo 100% Pure Coconut Oil, 600 ml  Coconut4.3 out of 5 stars 115  700+ viewed in past weekLimited time deal  ₹120₹120 (₹200/l) M.R.P: ₹249₹249  (52% off)Buy 3 items, get 3% off
Amazon Brand - Solimo 100% Pure Coconut Oil, 1 L  Coconut4.2 out of 5 stars 165  300+ viewed in past weekLimited time deal  ₹225₹225 (₹225/

In [4]:
data_test  = scrape_amazon_data("handbag")

In [ ]:
data_test